## Ноут-шаблон под обучение моделей
> **Важно: обязательно фиксируем рандом (`seed=42`) для всех библиотек и методов, где под капотом подразумевается рандом!** 

> **Почему это важно:** для гарантии консистентности данных при сравнении разных моделей и решений.


In [268]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.models import FastText


from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC


import random
import ast

# фиксирую то, что импортнул
random.seed(42)
np.random.seed(42)

In [28]:
# Скачиваем необходимые ресурсы (выполнить один раз)
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

### Импорт датасета и сплит на выборки

In [98]:
# Загружаем датасет
df = pd.read_csv("customer_support_dataset_generated.csv")
df.head()

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [108]:
df.loc[28120]

flags                                                        BLC
instruction    I want to create an account and also see my de...
category                                       ACCOUNT, DELIVERY
intent                          create_account, delivery_options
response       Certainly! To create a new account, please hea...
Name: 28120, dtype: object

### Предобработка

In [182]:
# Обязательный блок: исправляем дублящиеся категории и интенты
def process_category_unique(value):
    if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        try:
            categories = ast.literal_eval(value)
            unique_categories = list(set(categories))
            # Если осталась одна категория - возвращаем как строку
            if len(unique_categories) == 1:
                return unique_categories[0]
            else:
                return ', '.join(sorted(unique_categories))
        except:
            return value
    else:
        return value

    
    
df['category'] = df['category'].apply(process_category_unique).astype(str)
df['intent'] = df['intent'].apply(process_category_unique).astype(str)

In [183]:
df.loc[28120]

flags                                                              BLC
instruction          I want to create an account and also see my de...
category                                             ACCOUNT, DELIVERY
intent                                create_account, delivery_options
response             Certainly! To create a new account, please hea...
clean_instruction         want create account also see delivery option
Name: 28120, dtype: object

**Дальше я делаю подготовку текстовок обращений**

В целом, этот блок на ваше усмотрение можно кастомить, если я что-то забыл учесть, или у вас появилась классная идея, ~~или я где-то жестоко не прав и ошибся~~, но базовый минимум здесь - дропнуть английские стоп-слова и пунктуацию, привести слова к леммам

In [184]:
# Так было в моём EDA

# stop_words = set(stopwords.words('english'))  # или 'russian', 'spanish' и т.д.


# def remove_stopwords(text):
#     # Удаляем различные шаблоны с фигурными скобками
#     patterns_to_remove = [r'{{.*?}}']
    
#     for pattern in patterns_to_remove:
#         text = re.sub(pattern, '', text)
#     tokens = word_tokenize(text.lower())
#     tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
#     return " ".join(tokens)


# df['clean_instruction'] = df['instruction'].apply(remove_stopwords)

In [185]:
class TextPreprocessor:
    def __init__(self, language='english', use_lemmatization=True, custom_stopwords=None):
        
        self.stop_words = set(stopwords.words(language))
        if custom_stopwords:
            self.stop_words.update(custom_stopwords)
        
        self.lemmatizer = WordNetLemmatizer() if use_lemmatization else None
        self.stemmer = PorterStemmer() if not use_lemmatization else None
        
        # Расширенные шаблоны для удаления
        self.patterns_to_remove = [
            r'{{.*?}}',  # шаблоны с фигурными скобками
            r'\[.*?\]',  # квадратные скобки
            r'<.*?>',    # HTML теги
            r'http\S+',  # URL
            r'@\w+',     # упоминания
            r'#\w+',     # хэштеги
        ]
    
    def clean_text(self, text):
        if not isinstance(text, str):
            return ""
        
        # Приведение к нижнему регистру
        text = text.lower()
        
        # Удаление шаблонов
        for pattern in self.patterns_to_remove:
            text = re.sub(pattern, '', text)
        
        # Удаление пунктуации и цифр
        text = text.translate(str.maketrans('', '', string.punctuation + string.digits))
        
        # Токенизация
        tokens = word_tokenize(text)
        
        # Фильтрация и нормализация
        filtered_tokens = []
        for token in tokens:
            if (len(token) > 1 and                    # убираем одиночные символы
                token.isalpha() and                   # только буквы
                token not in self.stop_words):        # не стоп-слово
                
                if self.lemmatizer:
                    token = self.lemmatizer.lemmatize(token)
                elif self.stemmer:
                    token = self.stemmer.stem(token)
                
                filtered_tokens.append(token)
        
        return " ".join(filtered_tokens)


In [186]:
# Пример использования
# Создаем препроцессор
preprocessor = TextPreprocessor(language='english', use_lemmatization=True)

df['clean_instruction'] = df['instruction'].apply(preprocessor.clean_text)

In [187]:
preprocessor.stop_words # для понимания, что там лежит 

# no, not -> вероятно не нужно исключать, подумать
# i do **not** want to cancel order, i want to change shipping address

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [188]:
df[['instruction', 'clean_instruction']].sample(25)

,instruction,clean_instruction
4513,I do not know how to check the bills from {{Pe...,know check bill
30359,I need to begin the process of creating an acc...,need begin process creating account
17043,I need assistance to sign up to your newsletter,need assistance sign newsletter
20542,can you help me to restore the pass of my acco...,help restore pas account
715,i need help canceling order {{Order Number}},need help canceling order
7119,how to lodge a reclamation against your company?,lodge reclamation company
21008,i cant sign up i need to inform of signup erroprs,cant sign need inform signup erroprs
40362,I would like to place an order for some flowers.,would like place order flower
24452,i want assistance using tghe fucking {{Account...,want assistance using tghe fucking account
24274,can you help me using the premium profile?,help using premium profile


### Сплит выборки
Разбиваем на:
- `train` - на нём обучаем модели
- `validation` - на нём "тюним" модели, настраимаем гиперпараметры (когда GridSearch делаем, к примеру)
- `test` - финальная выборка для оценки качества  Тестовый набор используется **ТОЛЬКО ОДИН РАЗ** в самом конце! При обучении и подборе параметров модель ничего не должна знать из этого датасета.

Предлагаю бить в пропорции **70/15/15**

- train
- validation для экспериментов
- test откладываем и забываем - его доразбить для имитации сервиса

In [189]:
def train_val_test_split(X, 
                         y, 
                         val_size=0.15, 
                         test_size=0.15, 
                         random_state=42 # !!!
                         ):
    """
    Разделяет данные на train, validation и test наборы
    """

    # Сначала отделяем test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, 
        y, 
        test_size=test_size, 
        random_state=random_state, 
        shuffle=True
        # stratify=y  # есть классы всего лишь с 1 примером, поэтому просто шафлим
    )
    
    # Затем разделяем на train и validation
    relative_val_size = val_size / (1 - test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, 
        y_train_val, 
        test_size=relative_val_size, 
        random_state=random_state, 
        shuffle=True
        # stratify=y_train_val  # есть классы всего лишь с 1 примером, поэтому просто шафлим
    )
    
    return X_train, X_val, X_test, y_train, y_val, y_test



# Получаем выборки
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    X=df.drop(['intent'], axis=1), 
    y=df[['intent']]
)

In [313]:
df.shape, X_train.shape, X_val.shape, X_test.shape

((40366, 6), (28256, 5), (6055, 5), (6055, 5))

### Векторизация
Пробуем следующие методы:
- Bag of Words (мешок слов)
- TF-IDF
- Word2Vec
- FastText

Статья, которой можно вдохновиться: https://habr.com/ru/articles/778048/

#### Bag of Words

In [36]:
# Bag of Words

vec_bag = CountVectorizer()
X_train_bag = vec_bag.fit_transform(X_train['clean_instruction'])
X_val_bag = vec_bag.transform(X_val['clean_instruction']) 

# Результаты
print(pd.DataFrame(X_train_bag.toarray(), columns=vec_bag_train.get_feature_names_out()))

       aa  aaa  aabout  aaccount  aailable  aasistance  aat  abailable  abc  \
0       0    0       0         0         0           0    0          0    0   
1       0    0       0         0         0           0    0          0    0   
2       0    0       0         0         0           0    0          0    0   
3       0    0       0         0         0           0    0          0    0   
4       0    0       0         0         0           0    0          0    0   
...    ..  ...     ...       ...       ...         ...  ...        ...  ...   
28251   0    0       0         0         0           0    0          0    0   
28252   0    0       0         0         0           0    0          0    0   
28253   0    0       0         0         0           0    0          0    0   
28254   0    0       0         0         0           0    0          0    0   
28255   0    0       0         0         0           0    0          0    0   

       abcxyz  ...  yor  yoru  youe  youhave  youhe

#### TF-IDF

In [37]:
# TF-IDF

vec_tfidf = TfidfVectorizer()
X_train_tfidf = vec_tfidf.fit_transform(X_train['clean_instruction'])
X_val_tfidf = vec_tfidf.transform(X_val['clean_instruction'])

# Результаты
print(pd.DataFrame(X_train_tfidf.toarray(), columns=vec_tfidf_train.get_feature_names_out()))

        aa  aaa  aabout  aaccount  aailable  aasistance  aat  abailable  abc  \
0      0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
1      0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
2      0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
3      0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
4      0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
...    ...  ...     ...       ...       ...         ...  ...        ...  ...   
28251  0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
28252  0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
28253  0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
28254  0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   
28255  0.0  0.0     0.0       0.0       0.0         0.0  0.0        0.0  0.0   

       abcxyz  ...  yor  yoru  youe  yo

#### Word2Vec

In [84]:
# Word2Vec

X_train_tokenized_docs = [word_tokenize(doc) for doc in X_train['clean_instruction']]
X_val_tokenized_docs = [word_tokenize(doc) for doc in X_val['clean_instruction']]

word2vec_model = Word2Vec(
    sentences=X_train_tokenized_docs,
    vector_size=100,     # размер вектора
    window=5,            # размер окна контекста
    min_count=1,         # минимальная частота слова
    workers=1,           # количество потоков, 1 - иначе рандомит
    sg=1                 # 1 для skip-gram, 0 для CBOW
)

word2vec_model.train(X_train_tokenized_docs, total_examples=len(X_train_tokenized_docs), epochs=10)

# взять предобученный w2v

(765618, 1170580)

In [85]:
# Получение векторов слов
print("Вектор для слова 'help':")
print(word2vec_model.wv['help'])

# Похожие слова
print("\nПохожие слова на 'help':")
print(word2vec_model.wv.most_similar('help', topn=10))

Вектор для слова 'help':
[-0.35163915  0.31268397  0.49166417  0.18095809 -0.4500031  -0.19039911
  0.4859719   0.28821427 -0.42653376 -0.56383204 -0.02173221 -0.35532773
 -0.29702744  0.46302262 -0.09049392  0.10901327  0.00952306 -0.36831054
 -0.35835877 -0.48667234 -0.02569739 -0.4558875   0.56171143 -0.41802764
 -0.07442138 -0.12876812 -0.08502872 -0.17657785 -0.20255297  0.01478856
  0.37100756 -0.61568016 -0.33489096 -0.6893172   0.02288232  0.18579105
  0.80613977  0.52543247  0.00703939 -0.48679838 -0.1365609   0.05675988
 -0.69686127  0.03633102  0.11787723 -0.11330228 -0.22351696  0.74777347
  0.6143787   0.62323964  0.17270592  0.09202094  0.14689662 -0.05326167
 -0.09115104  0.44239253  0.33902174 -0.09935383 -0.4633681   0.14980492
  0.6583257  -0.39091736  0.47928745  0.04092602 -0.02594467  0.6327843
 -0.5635424   0.3107693  -0.5906118   0.3296252   0.16477898  0.58776623
 -0.07015681  0.25954866  0.0489395  -0.37317657 -0.4698028   0.24030699
  0.11085309 -0.02472962  0

In [88]:
# Векторное представление документа (среднее векторов слов)
def document_vector(model, doc):
    words = [word for word in doc if word in model.wv.key_to_index]
    if len(words) == 0:
        return np.zeros(model.vector_size)
    return np.mean(model.wv[words], axis=0)


X_train_w2v = [document_vector(word2vec_model, doc) for doc in X_train_tokenized_docs]
X_val_w2v = [document_vector(word2vec_model, doc) for doc in X_val_tokenized_docs]
print(f"Длина вектора: {X_train_w2v[0].shape}")

Длина вектора: (100,)


#### FastText

In [90]:
# FastText

fasttext_model = FastText(
    vector_size=100,
    window=5,
    min_count=1,
    workers=1,
    sg=1
)

# Обучение модели
fasttext_model.build_vocab(X_train_tokenized_docs)
fasttext_model.train(
    X_train_tokenized_docs, 
    total_examples=len(X_train_tokenized_docs), 
    epochs=10
)

# взять предобученный ft

(765259, 1170580)

In [91]:
# FastText может работать с OOV (out-of-vocabulary) словами
print("Вектор для существующего слова 'help':")
print(fasttext_model.wv['машинное'])

# Вектор для несуществующего слова (FastText использует n-grams)
fake_word = 'looooooooooooooool'
print(f"\nВектор для OOV слова '{fake_word}':")
print(fasttext_model.wv[fake_word])

# Похожие слова
print("\nПохожие слова на 'help':")
print(fasttext_model.wv.most_similar('help', topn=10))

Вектор для существующего слова 'help':
[ 1.11393468e-03  4.75587905e-04  1.19153515e-03 -2.63607886e-04
  1.30447920e-03  1.99535090e-04 -7.94179796e-04 -1.15824083e-03
  5.81545406e-04 -3.27856076e-04 -3.89009423e-04  4.12308174e-04
 -1.42313109e-03  6.28998620e-04 -5.55658131e-04  2.32535950e-03
 -1.15962466e-04 -1.49298401e-03 -1.12431333e-03 -1.35366514e-03
 -9.60248173e-04 -3.45842447e-04 -2.23360606e-03  1.28284283e-03
 -1.09591370e-03  9.07768437e-04  1.05625973e-03  5.91978314e-04
 -4.26866136e-05 -2.64778664e-05  9.93702561e-05 -1.89065689e-03
  1.66521022e-05  2.12211139e-03  8.87380273e-04 -2.13543564e-04
 -1.35282008e-03  1.62534649e-04  1.14424278e-04  6.93461916e-04
 -1.03792537e-03  1.88554055e-03  8.84957670e-04  9.07533686e-04
  7.69513543e-04 -4.26217332e-04 -4.96534922e-04 -1.33164623e-03
 -8.58099142e-04  1.87993236e-03 -3.83349339e-04 -2.88326666e-03
 -8.69264244e-04  7.40516407e-04 -4.80684230e-06 -1.38817274e-03
  9.47783352e-04 -1.19454646e-03  1.10136461e-03 -5

In [92]:
X_train_ft = [document_vector(fasttext_model, doc) for doc in X_train_tokenized_docs]
X_val_ft = [document_vector(fasttext_model, doc) for doc in X_val_tokenized_docs]
print(f"Длина вектора: {X_train_ft[0].shape}")

Длина вектора: (100,)


In [ ]:
# на подумать - пофиксить орфографию, стоит ли

#### Сравнялка

In [94]:
# from sklearn.metrics.pairwise import cosine_similarity

# def compare_methods(docs):
#     """Сравнение различных методов векторного представления"""
    
#     # 1. Bag of Words
#     bow_vectorizer = CountVectorizer()
#     bow_vectors = bow_vectorizer.fit_transform(docs)
    
#     # 2. TF-IDF
#     tfidf_vectorizer = TfidfVectorizer()
#     tfidf_vectors = tfidf_vectorizer.fit_transform(docs)
    
#     # 3. Word2Vec
#     tokenized_docs = [word_tokenize(doc.lower()) for doc in docs]
#     w2v_model = Word2Vec(sentences=tokenized_docs, vector_size=100, window=5, min_count=1, sg=1)
#     w2v_vectors = np.array([document_vector(w2v_model, doc) for doc in tokenized_docs])
    
#     # 4. FastText
#     ft_model = FastText(vector_size=100, window=5, min_count=1, sg=1)
#     ft_model.build_vocab(tokenized_docs)
#     ft_model.train(tokenized_docs, total_examples=len(tokenized_docs), epochs=10)
#     ft_vectors = np.array([document_vector(ft_model, doc) for doc in tokenized_docs])
    
#     # Сравнение косинусной схожести между документами
#     methods = {
#         'BoW': bow_vectors,
#         'TF-IDF': tfidf_vectors,
#         'Word2Vec': w2v_vectors,
#         'FastText': ft_vectors
#     }
    
#     print("Косинусная схожесть между документами:")
#     for method_name, vectors in methods.items():
#         if hasattr(vectors, 'toarray'):
#             vectors = vectors.toarray()
#         similarity = cosine_similarity(vectors)
#         print(f"\n{method_name}:")
#         print(similarity)

# # Запуск сравнения
# compare_methods(X_train['clean_instruction'])

Косинусная схожесть между документами:

BoW:
[[1.         0.23570226 0.         ... 0.         0.         0.        ]
 [0.23570226 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]

TF-IDF:
[[1.         0.13995187 0.         ... 0.         0.         0.        ]
 [0.13995187 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]

Word2Vec:
[[0.99999994 0.7913146  0.5672211  ... 0.587035

### Классификация интентов
**Модели**

Точно пробуем из линейных моделей SVM и логистическую регрессию, но в целом можно поэкспериментировать.

Для многоклассовой классификации - можно воспользоваться "One-vs-Rest" и "One-vs-One"

**Метрики** 

f1-score, precision и recall (micro, macro и weighted - всё имплементировано в sklearn'е)

Пара штрихов перед обучением

In [204]:
# для multilabeled - делаем, чтобы всё хранилось в виде списков

y_train['intent'] = y_train['intent'].apply(lambda x: x.split(', ') if isinstance(x, str) else [x])
y_val['intent'] = y_val['intent'].apply(lambda x: x.split(', ') if isinstance(x, str) else [x])
y_test['intent'] = y_test['intent'].apply(lambda x: x.split(', ') if isinstance(x, str) else [x])

In [233]:
# бинарайзер - для мультилейбл классификатора
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train['intent'])
y_val_bin = mlb.transform(y_val['intent'])
y_test_bin = mlb.transform(y_test['intent'])

y_train_bin

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [264]:
X_train_list = [X_train_bag.toarray(), X_train_tfidf.toarray(), X_train_w2v, X_train_ft]
X_val_list = [X_val_bag.toarray(), X_val_tfidf.toarray(), X_val_w2v, X_val_ft]

X_train_val_list = [[X_train_bag.toarray(), X_val_bag.toarray()], 
                    [X_train_tfidf.toarray(), X_val_tfidf.toarray()], 
                    [X_train_w2v, X_val_w2v], 
                    [X_train_ft, X_val_ft]
                   ]

# X_test_list = [X_test_bag.toarray(), X_test_tfidf.toarray(), X_test_w2v, X_test_ft]


#### LogisticRegression

In [266]:
logreg_list = []
y_pred_list = []

for x_train_val in X_train_val_list:
    # MultiOutputClassifier - обёртка для multilabeles классификации
    # пока сделаем с дефолтными параметрами
    model = MultiOutputClassifier(LogisticRegression(random_state=42))
    model.fit(x_train_val[0], y_train_bin)

    # Предсказания
    y_pred = model.predict(x_train_val[1])
    
    # Сохранили модель и предикт
    logreg_list.append(model)
    y_pred_list.append(y_pred)

/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

In [267]:
logreg_list = []
y_pred_list = []

for x_train_val in X_train_val_list:
    # MultiOutputClassifier - обёртка для multilabel
    # поменяли solver
    model = MultiOutputClassifier(LogisticRegression(solver='saga', random_state=42))
    model.fit(x_train_val[0], y_train_bin)

    # Предсказания
    y_pred = model.predict(x_train_val[1])
    
    # Сохранили модель и предикт
    logreg_list.append(model)
    y_pred_list.append(y_pred)
    
# обучалось минут 15 :( и всё равно не сошлось

/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/vadim/opt/anaconda3/lib/python3.9/site-pack

In [275]:
vecs = ['BoW', 'TF-IDF', 'W2V', 'FT']
for i in range(4):
    print(vecs[i])
    print("Micro F1:", f1_score(y_val_bin, y_pred_list[i], average='micro'))
    print("Macro F1:", f1_score(y_val_bin, y_pred_list[i], average='macro')) 
    print("Weighted F1:", f1_score(y_val_bin, y_pred_list[i], average='weighted'))
    print(classification_report(y_val_bin, y_pred_list[i]))
    print('-'*10)

    
# модель предсказала все нули???
# стоит ли сделать , zero_division=0?

BoW
Micro F1: 0.945344129554656
Macro F1: 0.9449981512441258
Weighted F1: 0.9448252212982081
              precision    recall  f1-score   support

           0       0.97      0.91      0.94       228
           1       0.99      0.93      0.96       183
           2       0.97      0.94      0.95       228
           3       1.00      0.96      0.98       226
           4       0.94      0.89      0.92       246
           5       0.99      0.94      0.96       231
           6       0.98      0.91      0.95       257
           7       1.00      0.89      0.94       222
           8       0.98      0.91      0.94       205
           9       0.94      0.94      0.94       254
          10       0.96      0.90      0.93       226
          11       0.99      0.96      0.97       263
          12       0.96      0.86      0.91       249
          13       0.98      0.92      0.95       216
          14       0.97      0.87      0.92       206
          15       0.95      0.85      0.9

/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vadim/opt/an

### LinearSVC

In [305]:
# LinearSVC

svc_list = []
y_svc_pred_list = []

for x_train_val in X_train_val_list:
    # MultiOutputClassifier - обёртка для multilabel классификации
    # поменяли solver
    model = MultiOutputClassifier(LinearSVC(random_state=42))
    model.fit(x_train_val[0], y_train_bin)

    # Предсказания
    y_pred = model.predict(x_train_val[1])
    
    # Сохранили модель и предикт
    svc_list.append(model)
    y_svc_pred_list.append(y_pred)
    


In [306]:
vecs = ['BoW', 'TF-IDF', 'W2V', 'FT']
for i in range(4):
    print(vecs[i])
    print("Micro F1:", f1_score(y_val_bin, y_svc_pred_list[i], average='micro'))
    print("Macro F1:", f1_score(y_val_bin, y_svc_pred_list[i], average='macro')) 
    print("Weighted F1:", f1_score(y_val_bin, y_svc_pred_list[i], average='weighted'))
    print(classification_report(y_val_bin, y_svc_pred_list[i]))
    print('-'*10)

BoW
Micro F1: 0.9628712871287128
Macro F1: 0.9627959044823583
Weighted F1: 0.9627172534891602
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       228
           1       0.97      0.95      0.96       183
           2       0.97      0.96      0.96       228
           3       0.99      0.99      0.99       226
           4       0.93      0.93      0.93       246
           5       0.99      0.98      0.98       231
           6       0.98      0.94      0.96       257
           7       1.00      0.95      0.97       222
           8       0.96      0.91      0.93       205
           9       0.93      0.94      0.94       254
          10       0.96      0.95      0.96       226
          11       0.99      0.98      0.99       263
          12       0.93      0.95      0.94       249
          13       0.97      0.95      0.96       216
          14       0.96      0.95      0.95       206
          15       0.96      0.89      0.

/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vadim/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vadim/opt/an

## Вопросы на подумать
- **подозрительно хороший скор** - кажется, я где-то жестоко неправ
- мультикласс - в нашем случае лучше onehotencode'ить каждую вариацию? или рассматривать комбинацию интентов как отдельный класс? unique-значения и играться порогами для моделей, чтобы было 1-2 метки предсказанных
- как бы чистить выбросы типо "question abobut, aaaaabout" и прочее? или их не надо чистить, считаем это нормальным шумом, который вполне реален? не теряет ли модель качество в этом шуме?
- ~~надо преобразование для трейна и валидации делать одной или разными векторайзерами? по логике кажется, что одним~~
- **пороги вероятностей для моделей**

### Мысли
- не сходится логрег и оч долго обучается - что-то странное
- точно нужно попробовать подбор гиперпараметров, с помощью GridSearchCV к примеру
- модель предсказала все нули??? стоит ли сделать , zero_division=0?

- думаем про стоп-слова и комменты повыше
- можно подумать над доп. метриками (взвешенный accuracy, кастомные)
- перебераем параметры, смотрим на метрики, мб графики
- попробовать ещё моделей
- пайплайн обучения, пайплайн применения - всё объединить

In [302]:
y_pred_list[0][np.sum(y_pred_list[0], axis=1) == 0][0], 
y_pred_list[0][np.sum(y_pred_list[0], axis=1) == 0].shape

(400, 27)

In [311]:
y_pred_list[0][np.sum(y_pred_list[0], axis=1) == 2].shape

(91, 27)

In [307]:
y_svc_pred_list[0][np.sum(y_svc_pred_list[0], axis=1) == 0][0], 
y_svc_pred_list[0][np.sum(y_svc_pred_list[0], axis=1) == 0].shape

(164, 27)

In [312]:
y_svc_pred_list[0][np.sum(y_svc_pred_list[0], axis=1) == 2].shape

(121, 27)

In [318]:
y_pred_list[0][np.sum(y_pred_list[0], axis=1) >= 2][0]

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

MultiOutputClassifier(estimator=LinearSVC(random_state=42))